In [1]:
:opt no-lint
{-# LANGUAGE ScopedTypeVariables TypeApplications RankNTypes #-}

## 계산이 끝나지 않는 람다식
람다식 작은걸음 $(\lambda x.x\;x)\;e_2 \longmapsto e_2\;e_2$를 생각해 보자.
여기서 $e_2$가 적용하는 함수 $(\lambda x.x\;x)$와 똑같은 식인 경우에 어떤 일이 벌어질까?
바로 아래와 같이 된다.
\newline\indent
$(\lambda x.x\;x)(\lambda x.x\;x) \longmapsto (\lambda x.x\;x)(\lambda x.x\;x) \longmapsto (\lambda x.x\;x)(\lambda x.x\;x) \longmapsto \ldots$
\newline
$\beta$규칙에 따라 함수 몸체 $x\;x$에서 $x$를 $(\lambda x.x\;x)$로 바꿔치면
원래 처음 시작했던 식인 $(\lambda x.x\;x)(\lambda x.x\;x)$가 되어버린다.
그래서 아무리 여러 번 $\beta$줄임을 반복해도 제자리걸음만 할 뿐 계산이 끝나지 않는다.
이제 드브루인 인덱스(그림 \ref{fig:ChurchDeBruijn})를 활용해 타입없는 람다계산법에서 끝나지
않는 계산을 하스켈로 다뤄보자. 우선 람다계산법의 문법구조를 아래와 같은
데이터 타입으로 선언한다.

In [2]:
data DExpr = DI Int | DAbs DExpr | DApp DExpr DExpr deriving (Eq,Ord)

In [3]:
-- GHC가 자동으로 deriving하는 Show 인스턴스가 아닌  더 간략한 출력을 위해
instance Show DExpr where   -- 직접 작성한 DExpr에 대한 Show 인스턴스
  showsPrec _ (DI i) = showString (show i)
  showsPrec p (DAbs e) = showParen (p > 1) $
              showString "\\" . showsPrec 1 e
  showsPrec p (DApp e1 e2) = showParen (p > 9) $
              showsPrec 9 e1 . showString " " . showsPrec 10 e2

In [4]:
DAbs (DAbs (DI 1 `DApp` DI 2))        -- \x.\y.x y
DAbs (DI 1) `DApp` DAbs (DI 1)        -- (\x.x)(\x.x)
DAbs ((DI 1 `DApp` DI 1) `DApp` DI 1) -- \x.x x x
DAbs (DI 1 `DApp` (DI 1 `DApp` DI 1)) -- \x.x (x x)

\\1 2

(\1) (\1)

\1 1 1

\1 (1 1)

\noindent
그리고 매개변수에 인자를 넘겨받아 자기 자신에 적용하는 (self-appliation) 함수 $\lambda x.x\;x$는
드브로인 인덱스를 활용한 표현으로는 $\lambda 1~1$이다. 이를 `w`로 선언해 놓고 앞으로 의미구조를
하스켈로 구현하여 `w w`가 제자리걸음함을 확인해 보자.

In [5]:
w = DAbs (DI 1 `DApp` DI 1) -- \x.x x

w         -- self-application 함수인 w
DApp w w  -- w를 자기 자신에 적용한 식

\1 1

(\1 1) (\1 1)

작은걸음 의미구조는 $\beta$규칙 및 맥락 규칙들로 정의할 수 있다.
\newline\vspace*{-2ex}
$$(\lambda\,e)\;e_2 \longmapsto \;\downarrow_0^1 \{1{\mapsto}(\uparrow_0^1 e_2)\}e$$
$$\frac{~ e\longmapsto e' ~}{~ \lambda\,e \longmapsto \lambda\,e' ~}
\qquad\frac{~ e_1\longmapsto e_1'~}{~ e_1~e_2\longmapsto e_1'~e_2 ~}
\qquad\frac{~ e_2\longmapsto e_2'~}{~ e_1~e_2\longmapsto e_1~e_2' ~}
$$
위에서 $\downarrow_m^k$, $\uparrow_m^k$와 바꿔치기 $\{i{\mapsto}e\}$가 어떻게 동작하는지만 확실히 하면
나머지 맥락 규칙에 대한 처리는 산술식 작은걸음 계산기인 `step`을 작성하듯 선언하면 된다.
$\downarrow_m^k = \uparrow_m^{-k}$이며, 일반적으로 $\downarrow_m^k e$는 식 $e$에서
$m$보다 큰 드브루인 인덱스를 $k$만큼씩 증가시킨 식을 나타내는 표현으로 왼쪽 아래처럼 정의할 수 있다.
함수요약식에 대한 정의에서 함수 전체에는 $m$이다가 함수 몸체에서 $m+1$이
되는 이유는 바인더 $\lambda$가 한겹 더 감싸진 몸체 안에서는 그 바깥에서와
같은 대상을 나타내는 드브루인 인덱스가 1씩  더 큰 수가 되기 때문이다.
바꿔치기 $\{i{\mapsto}e\}$는 $\uparrow_m^k$ 표현을 활용해 오른쪽 아래와 같이 정의할 수 있다.
$$\begin{array}{ll}
\uparrow_m^k i 
\!\!\!\!&= \begin{cases}i+k & (i>m) \\ i & (i\le m)\end{cases} \\
\uparrow_m^k (\lambda\,e)
\!\!\!\!&=~ \lambda\,(\uparrow_{m{+}1}^k e) \\
\uparrow_m^k (e_1\;e_2)
\!\!\!\!&=~ (\uparrow_m^k e_1)\;(\uparrow_m^k e_2)
\end{array}
\qquad~~
\begin{array}{ll}
\{i{\mapsto}e\}j 
\!\!\!\!&= \begin{cases}e & (i=j) \\ j & (i\neq j)\end{cases}\\
\{i{\mapsto}e\}(\lambda\, e_1) 
\!\!\!\!&=~ \lambda\, \left\{(i{+}1){\mapsto}(\uparrow_0^1 e\right)\}e_1 \\
\{i{\mapsto}e\}(e_1\;e_2)
\!\!\!\!&=~ \{i{\mapsto}e\}e_1~\{i{\mapsto}e\}e_2 \\
\end{array}$$

\vspace*{-2ex}

In [6]:
up m k (DI i)       = DI (if i>m then i+k else i)
up m k (DAbs e)     = DAbs (up (m+1) k e)
up m k (DApp e1 e2) = DApp (up m k e1) (up m k e2)

dn m k = up m (-k)

subst i e (DI j)       = if i==j then e else DI j
subst i e (DAbs e1)    = DAbs (subst (i+1) (up 0 1 e) e1) 
subst i e (DApp e1 e2) = DApp (subst i e e1) (subst i e e2)

In [7]:
:type up
:type subst

up :: Int -> Int -> DExpr -> DExpr

subst :: Int -> DExpr -> DExpr -> DExpr

In [8]:
beta :: DExpr -> [DExpr]  -- 일단 베타 규칙만 작성
beta (DApp (DAbs e) e2) = [ dn 0 1 (subst 1 (up 0 1 e2) e) ]
beta _                  = []

In [10]:
DApp w w
beta (DApp w w) -- 제자리걸음 확인!

(\1 1) (\1 1)

[(\1 1) (\1 1)]

In [9]:
step :: DExpr -> [DExpr]
step (DI _)       = []
step (DApp e1 e2) = beta (DApp e1 e2)
                 ++ [DApp e1' e2 | e1' <- step e1]
                 ++ [DApp e1 e2' | e2' <- step e2]
step (DAbs e)     = [DAbs e' | e' <- step e]

In [18]:
-- step이 제대로 동작하는지 다음 내용으로 테스트해 보라
eee1 = DAbs . DAbs $ DI 1 `DApp` DI 2 `DApp` DI 3
eee2 = DAbs . DAbs $ DI 1 `DApp` DI 3
DAbs $ DApp eee1 eee2 -- \x.(\y.\z.z y x) (\w.\v.v x)
step it               -- \x.\y.\z.z (\w.\v x) x

\(\\1 2 3) (\\1 3)

[\\1 (\\1 4) 2]

## 부동점(fixpoint)

In [12]:
yC = DAbs (DApp e211 e211) -- \f.(\x.f(x x))(\x.f(x x))
   where e211 = DAbs (DApp (DI 2) (DI 1 `DApp` DI 1))

In [13]:
import Data.List ()
f = head
yC `DApp` (DAbs $ DI 1)
step $ yC `DApp` (DAbs $ DI 1)
step $ head it
step $ it!!1
step $ last it
step $ head it
step $ head it

(\(\2 (1 1)) (\2 (1 1))) (\1)

[(\(\1) (1 1)) (\(\1) (1 1)),(\1 ((\2 (1 1)) (\2 (1 1)))) (\1)]

[(\1) ((\(\1) (1 1)) (\(\1) (1 1))),(\1 1) (\(\1) (1 1)),(\(\1) (1 1)) (\1 1)]

[(\(\1) (1 1)) (\(\1) (1 1)),(\1 1) (\1 1)]

[(\1 1) (\1 1)]

[(\1 1) (\1 1)]

[(\1 1) (\1 1)]

# 다형타입 람다계산법
\label{sec:SystemF}

문법구조\vspace{-4.5ex}
\begin{align*}
\qquad
t \in B & ~\text{\small(기본타입)} &
\tau\in T ~::= ~& t ~\mid~ \tau \to \tau
~\mid~ \alpha ~\mid~ \forall \alpha.\tau
\\
c \in C & ~\text{\small(기본상수)} &
e\in E ~::=~ & c \mid x \mid \lambda x{\,:\,}\tau.\,e \mid e~e
\mid \Lambda \alpha.\,e \mid e\,@\tau
\end{align*} 타입규칙
$\qquad
\Delta ~::=~ c{\,:\,}\tau,\Delta \mid \bm{\cdot} \qquad\qquad\qquad
\Gamma ~::=~ \alpha{\,:\,}\star,\Delta \mid x{\,:\,}\tau,\Delta \mid \bm{\cdot}$
$$
{\scriptstyle(\textsc{Con})}
\frac{~ ~}{~ \Gamma \vdash c:\tau ~}
({\scriptstyle c{\,:\,}\tau\;\in\,\Delta})
\qquad\qquad
{\scriptstyle(\textsc{Var})}
\frac{~ ~}{~ \Gamma \vdash x:\tau ~}
({\scriptstyle x{\,:\,}\tau\;\in\,\Gamma})
$$ \vspace*{-.5ex}
$$
{\scriptstyle(\textsc{Abs})}
\frac{~ x:\tau_2,\Gamma \vdash e:\tau ~}{~ \Gamma \vdash \lambda x{\,:\,}\tau_2.\,e : \tau_2\to\tau ~}
\qquad
{\scriptstyle(\textsc{App})}
\frac{~ \Gamma \vdash e_1:\tau_2\to\tau ~\quad~ \Gamma \vdash e_2:\tau_2 ~}{~ \Gamma \vdash e_1\;e_2 : \tau ~}
$$ \vspace*{-.5ex}
$$
{\scriptstyle(\textsc{TAbs})}
\frac{~ \alpha{\,:\,}\star,\Gamma \vdash e:\tau ~}{~ \Gamma \vdash \Lambda \alpha.\,e : \forall \alpha.\tau ~}
\qquad
{\scriptstyle(\textsc{TApp})}
\frac{~ \Gamma \vdash e:\forall\alpha.\tau ~}{~ \Gamma \vdash e\,@\tau_1 : \{\alpha{\mapsto}\tau_1\}\tau ~}
$$
~\newline
의미구조
$\quad (\lambda x{:}\tau.\,e)\,e_2 \longmapsto \{x{\mapsto}e_2\}e
\qquad (\Lambda \alpha.\,e)\,\tau \longmapsto \{\alpha{\mapsto}\tau\}e$
$$ \mathcal{E} ~::=~ \bullet
\mid \lambda x{\,:\,}\tau.\,\mathcal{E} 
\mid \mathcal{E}~e \mid e~\mathcal{E}
\mid \Lambda \alpha.\,\mathcal{E} \mid \mathcal{E}\,@\tau
\qquad
\frac{~e\longmapsto e'~}{~\mathcal{E}[e]\xmapsto{~_C~}\mathcal{E}[e']~}
$$

TODO

In [14]:
type BOOL = forall a. a -> a -> a 
false, true :: BOOL
false = \_ -> \y -> y
true  = \x -> \_ -> x

In [15]:
if_then_else_ 

: 

In [16]:
:type id

id :: forall a. a -> a

In [17]:
:type id @Int

id @Int :: Int -> Int

\section*{탐구과제}
1. TODO
1. TODO